<a href="https://colab.research.google.com/github/Alejandro-Casanova/pytorch-deep-learning/blob/main/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [17]:
%%writefile get_data.py
# YOUR CODE HERE
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"
train_path = image_path / "train"
test_path = image_path / "test"

def download_data():
  # If the image folder doesn't exist, download it and prepare it...
  if image_path.is_dir():
      print(f"{image_path} directory exists.")
      if train_path.is_dir() and test_path.is_dir():
        print("Data already downloaded and unzipped, exiting...")
        return
  else:
      print(f"Did not find {image_path} directory, creating one...")
      image_path.mkdir(parents=True, exist_ok=True)

  zip_path = data_path / "pizza_steak_sushi.zip"
  # Download pizza, steak, sushi data
  with open(zip_path, "wb") as f:
      request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
      print("Downloading pizza, steak, sushi data...")
      f.write(request.content)

  # Unzip pizza, steak, sushi data
  with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
      print("Unzipping pizza, steak, sushi data...")
      zip_ref.extractall(image_path)

  # Remove zip file
  os.remove(data_path / "pizza_steak_sushi.zip")

download_data()

Overwriting get_data.py


In [18]:
# Example running of get_data.py
!python get_data.py

data/pizza_steak_sushi directory exists.
Data already downloaded and unzipped, exiting...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [32]:
import requests
from pathlib import Path

script1 = "model_builder.py"
script1_raw_url = "https://raw.githubusercontent.com/Alejandro-Casanova/pytorch-deep-learning/main/going_modular/going_modular/model_builder.py"
script2 = "utils.py"
script2_raw_url = "https://raw.githubusercontent.com/Alejandro-Casanova/pytorch-deep-learning/main/going_modular/going_modular/utils.py"
script3 = "engine.py"
script3_raw_url = "https://raw.githubusercontent.com/Alejandro-Casanova/pytorch-deep-learning/main/going_modular/going_modular/engine.py"
script4 = "data_setup.py"
script4_raw_url = "https://raw.githubusercontent.com/Alejandro-Casanova/pytorch-deep-learning/main/going_modular/going_modular/data_setup.py"

def downloadScript(name: str, raw_url: str):
  # Download helper functions from Learn PyTorch repo (if not already downloaded)
  if Path(name).is_file():
    print(f"{name} already exists, skipping download")
  else:
    print(f"Downloading {name}")
    # Note: you need the "raw" GitHub URL for this to work
    request = requests.get(raw_url)
    with open(name, "wb") as f:
      f.write(request.content)


downloadScript(script1, script1_raw_url)
downloadScript(script2, script2_raw_url)
downloadScript(script3, script3_raw_url)
downloadScript(script4, script4_raw_url)

model_builder.py already exists, skipping download
utils.py already exists, skipping download
engine.py already exists, skipping download


In [47]:
%%writefile train.py
# YOUR CODE HERE
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils
import argparse
from torchvision import transforms

parser = argparse.ArgumentParser(
                    prog='Train',
                    description='Train Neural Network according to passed parameters',
                    epilog='by Alejandro Casanova')

parser.add_argument('-e', '--epochs', default=5, help="Number of epochs to train the model.")
parser.add_argument('-b', '--batch_size', default=32, help="Size of the batches data is divided in.")
parser.add_argument('-u', '--hidden_units', default=10, help="Number of hidden units each layer of the model has.")
parser.add_argument('-l', '--learning_rate', default=0.001, help="Learning rate for the optimizer.")

args = parser.parse_args()
print(args)

# Setup hyperparameters
NUM_EPOCHS = int(args.epochs)
BATCH_SIZE = int(args.batch_size)
HIDDEN_UNITS = int(args.hidden_units)
LEARNING_RATE = float(args.learning_rate)

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")


Overwriting train.py


In [42]:
# Example running of train.py
!python train.py --epochs 10 --batch_size 32 --hidden_units 10 --learning_rate 0.001

Namespace(epochs='10', batch_size='32', hidden_units='10', learning_rate='0.001')
  0% 0/10 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1115 | train_acc: 0.2734 | test_loss: 1.1094 | test_acc: 0.1979
 10% 1/10 [00:02<00:25,  2.81s/it]Epoch: 2 | train_loss: 1.0958 | train_acc: 0.4336 | test_loss: 1.1069 | test_acc: 0.2604
 20% 2/10 [00:04<00:16,  2.01s/it]Epoch: 3 | train_loss: 1.0940 | train_acc: 0.3086 | test_loss: 1.1109 | test_acc: 0.2917
 30% 3/10 [00:06<00:13,  1.91s/it]Epoch: 4 | train_loss: 1.0763 | train_acc: 0.5195 | test_loss: 1.1310 | test_acc: 0.1979
 40% 4/10 [00:08<00:12,  2.05s/it]Epoch: 5 | train_loss: 1.0404 | train_acc: 0.4141 | test_loss: 1.1391 | test_acc: 0.1979
 50% 5/10 [00:09<00:09,  1.84s/it]Epoch: 6 | train_loss: 1.0180 | train_acc: 0.4531 | test_loss: 1.1056 | test_acc: 0.2396
 60% 6/10 [00:11<00:06,  1.72s/it]Epoch: 7 | train_loss: 1.0269 | train_acc: 0.4023 | test_loss: 1.0866 | test_acc: 0.3220
 70% 7/10 [00:12<00:04,  1.64s/it]Epoch: 8 | train_loss: 0.9563 |

## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [65]:
%%writefile predict.py
from typing import List
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from model_builder import TinyVGG
import torch
import argparse
import data_setup

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"
class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=None,
    batch_size=32
)[2]

# Create transforms
pred_transform = transforms.Compose([
  transforms.Resize((64, 64))
])

# YOUR CODE HERE
def pred_and_plot_image(model: torch.nn.Module,
                        image_path: str,
                        class_names: List[str] = None,
                        transform=None,
                        device: torch.device = device):
    """Makes a prediction on a target image and plots the image with its prediction."""

    # 1. Load in image and convert the tensor values to float32
    target_image = torchvision.io.read_image(str(image_path)).type(torch.float32)

    # 2. Divide the image pixel values by 255 to get them between [0, 1]
    target_image = target_image / 255.

    # 3. Transform if necessary
    if transform:
        target_image = transform(target_image)

    # 4. Make sure the model is on the target device
    model.to(device)

    # 5. Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
        # Add an extra dimension to the image
        target_image = target_image.unsqueeze(dim=0)

        # Make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(target_image.to(device))

    # 6. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # 7. Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    print(f"Pred: {class_names[target_image_pred_label.cpu()]} | Prob: {target_image_pred_probs.max().cpu():.3f}")

    # 8. Plot the image alongside the prediction and prediction probability
    plt.imshow(target_image.squeeze().permute(1, 2, 0)) # make sure it's the right size for matplotlib
    if class_names:
        title = f"Pred: {class_names[target_image_pred_label.cpu()]} | Prob: {target_image_pred_probs.max().cpu():.3f}"
    else:
        title = f"Pred: {target_image_pred_label} | Prob: {target_image_pred_probs.max().cpu():.3f}"
    plt.title(title)
    plt.axis(False);


parser = argparse.ArgumentParser(
                    prog='Predict',
                    description='Use a model to predict on an image',
                    epilog='by Alejandro Casanova')

parser.add_argument('-i', '--image', type=str, default="data/pizza_steak_sushi/test/sushi/175783.jpg", help="Image to make prediction on")
parser.add_argument('-m', '--model_path', type=str, default="models/05_going_modular_script_mode_tinyvgg_model.pth", help="Model to use")

args = parser.parse_args()
print(args)

loaded_model = TinyVGG(
  input_shape=3,
  hidden_units=10,
  output_shape=len(class_names)
)
loaded_model.load_state_dict(torch.load(f=args.model_path))

# Pred on our custom image
pred_and_plot_image(model=loaded_model,
                    image_path=args.image,
                    class_names=class_names,
                    transform=pred_transform,
                    device=device)

Overwriting predict.py


In [66]:
# Example running of predict.py
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

Namespace(image='data/pizza_steak_sushi/test/sushi/175783.jpg', model_path='models/05_going_modular_script_mode_tinyvgg_model.pth')
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Pred: sushi | Prob: 0.482
